In [3]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

# The idea

First, retrieve all scene objects' configuration and position. 

In [2]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [3]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

In [4]:
listazinha = get_scene_objects_info(client_id, scene_objects)

In [5]:
for element in listazinha:
    a = round(element['object_position'][0],2)
    b = round(element['object_position'][1],2)
    c = round(element['object_position'][2],2)
    print([a,b,c])

[-1.72, 0.25, 0.25]
[-0.35, 1.5, 0.5]
[-1.52, -1.22, 0.5]
[-0.15, -0.1, 0.25]
[1.4, -0.15, 0.5]
[-2.02, 1.74, 0.15]


In [6]:
info_list, robot_information = split_robot_from_info_list(listazinha,'dr20')

In [7]:
for element in info_list:
    print(element['object_name'])

Cuboid_0
Cuboid_1
Cuboid_2
Cuboid_3
Cuboid_4


we know there are 4 horizontal normal vectors. Two of them point to X and Y. The other two point to -X and -Y. 

How to know where X is pointing? Well, it is given by the Z Euler vector. 

Where does Y point to? to X+90º. And what about -X and -Y? We just need to add 180º to X and Y.

# Creating Configuration Map

1. Step
To orient the robot so its euler angles will be 0,0,0.
2. Step
For each obstacle, do the following:
- Map bounding box vertices (in object's reference frame to global reference frame)
- Test applicability using the normal's orientation (i think i'll need to check sine and cossine, or limit the orientation within [0,2pi] )
- Create Configuration Vertice 

In [8]:
handle_dr20 = get_object_handle(client_id,'dr20')
euler_orientation = (0,0,0)
sim.simxSetObjectOrientation(client_id,handle_dr20,-1,euler_orientation,sim.simx_opmode_blocking)

0

In [9]:
robot_pos, robot_ang = get_configuration(client_id, handle_dr20)
180*robot_ang[2]/3.14

0.0332477198510318

In [10]:
get_normals_orientation(0)

(0, 1.570796326795, 3.14159265359, 4.7123889803850005)

In [16]:
for element in info_list:
    normals = get_normals_orientation(element['object_orientation'][2])
    # position_x = round
    print(f'{element["object_name"]}: {normals}')

Cuboid_0: (9.479927232965615e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_1: (6.891062969815315e-14, 1.5707963267950689, 3.141592653590069, 4.712388980385069)
Cuboid_2: (4.2405288371077856e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_3: (-1.7188674162509006e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_4: (2.1277126836615653e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)


## How to map the vertices to global reference frame?

Well, we have the euler angles, right? [We can create a rotation matrix using them](http://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf). Consider the angles $\phi$ (rotation around Z) , $\theta$ (rotation around Y) and $\psi$ (rotation around X). The rotation matrix is given by:

$$
R = R_z(\phi) R_y(\theta) R_x(\psi) = 
\begin{bmatrix}
cos \theta cos \phi & sin \psi sin \theta cos \phi - cos \psi sin \phi & cos \psi sin \theta cos \phi + sin \psi sin \phi \\
cos \theta sin \phi & sin \psi sin \theta sin \phi + cos \psi cos \phi & cos \psi sin \theta sin \phi - sin \psi cos \phi \\
-sin \theta & sin \psi cos \theta & cos \psi cos \theta \\
\end{bmatrix}.
$$

This means that, if we get a 3D point $\vec{x}$ and multiply by $R$, we get the same effect of rotating $\vec{x}$ $\phi$ around Z, $\theta$ around Y and $\psi$ around X. In order to get the point back to the original frame, we take the inverse of R.

In [7]:
import numpy as np
# testing the forward transform: vector = [1, 0, 0] and a rotation of 90º around Z axis
mat = np.round( rotation_matrix_from_euler_angles(3.14/2,0,0, False), 2 )
print(mat)
vector = np.array([1,0,0])
np.matmul(mat,vector) # vector goes to [0, 1, 0]

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [-0.  0.  1.]]


array([0., 1., 0.])

In [5]:
# testing the inverse: the vector is at [0, 1, 0], and we know there happened a Z axis 90º rotation. Where was the vector?
mat = np.round( rotation_matrix_from_euler_angles(3.14/2,0,0, True), 2 )
print(mat)
vector = np.array([0,1,0])
np.matmul(mat,vector) # obviously the vector was at [1, 0, 0]

[[ 0.  1.  0.]
 [-1.  0. -0.]
 [ 0.  0.  1.]]


array([1., 0., 0.])

In [9]:
# Ok, now a more case-specific test. Consider we have two reference frames, a global one and the robot's. 
# The only difference is that the robot's reference frame is rotated -90º around the Z axis. If a point on 
# robot's reference frame is equal to [1, 0, 0], where does it lie on the global reference frame?
# Naturally, we would say it lies on [0, -1, 0]. Which transformation should we use? Actually, the forward one.
 
mat = np.round( rotation_matrix_from_euler_angles(-3.14/2,0,0, False), 2 )
print(mat)
vector = np.array([1,0,0])
np.matmul(mat,vector) # see?

[[ 0.  1.  0.]
 [-1.  0. -0.]
 [-0.  0.  1.]]


array([ 0., -1.,  0.])